# Packages Installation(for colab)

In [ ]:
!pip3 install pycountry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 2.1 MB/s eta 0:00:00
Obtaining utils from git+https://github.com/gizdatalab/haystack_utils.git@main#egg=utils
  Cloning https://github.com/gizdatalab/haystack_utils.git (to revision main) to ./src/utils
  Running command git clone --filter=blob:none --quiet https://github.com/gizdatalab/haystack_utils.git /content/src/utils
  Resolved https://github.com/gizdatalab/haystack_utils.git to commit 087e5ae68b5eecbd4ca2415f522a0e8dfc61e617
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.1/713.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=138974fc8d73ea928a87ca74245e9b80d5181075340e56a913d94c31cf6073b1
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
import pandas as pd
import numpy as np
from typing import Callable, Dict, List, Optional, Text, Tuple, Union
import pycountry
from bs4 import BeautifulSoup
from IPython.display import display
import os

# Collect Responses

## Processing Functions

In [ ]:
def getCWResponse(path_to_cwdata:str)-> pd.DataFrame:
    """
    This function relies upon the "NDC Content" subsection of data from
    https://www.climatewatchdata.org/data-explorer/
    with 3 important files:
      1. CW_NDC_data_sector
      2. CW_NDC_metadata
      3. CW_NDC_tracker

    Params
    -----------
    path_to_cwdata: path to the NDC Content data from Climate Watch


    Return
    --------
    cw_data: Dataframe

    """
    # sector data file has the Climate watch response data
    data_sector  = pd.read_csv(path_to_cwdata+'CW_NDC_data_sector.csv')
    # we use the metadata to incorporate more info to response data
    ndc_metadata = pd.read_csv(path_to_cwdata+'CW_NDC_metadata.csv')
    tracker  = pd.read_csv(path_to_cwdata+'CW_NDC_tracker.csv')

    # Drop NA for country and question code
    cw_data = data_sector[data_sector.QuestionCode.notna()]
    cw_data = cw_data[cw_data.Country.notna()]
    cw_data = cw_data.reset_index(drop = True)
    cw_data['Sector'] = cw_data.Sector.str.strip()
    cw_data['SubSector'] = cw_data.SubSector.str.strip()
    cw_data['QuestionCode'] = cw_data.QuestionCode.str.strip()
    cw_data['ResponseText'] = cw_data.ResponseText.str.strip()
    # Getting metadata information
    cols_metadata = ndc_metadata[ndc_metadata.column_name != 'ghg_target_type'].set_index(
                                                    'column_name').T.to_dict('list')

    # Appending all info from metadata to the sector data dataframe
    cw_data['GlobalCategory'] =  cw_data['QuestionCode'].apply(
                                        lambda x: cols_metadata[x][0] if x in
                                        cols_metadata.keys() else None)
    cw_data['OverviewCategory'] =  cw_data['QuestionCode'].apply(
                                          lambda x: cols_metadata[x][1] if x in
                                          cols_metadata.keys() else None)

    cw_data['QuestionText'] =  cw_data['QuestionCode'].apply(lambda x:
                                    cols_metadata[x][3] if x in cols_metadata.keys()
                                      else None)
    cw_data['QuestionDefinition'] =  cw_data['QuestionCode'].apply(
                                              lambda x: cols_metadata[x][4] if x in
                                              cols_metadata.keys() else None)
    cw_data['GroupIndicator'] =  cw_data['QuestionCode'].apply(
                                        lambda x: cols_metadata[x][5] if x in
                                        cols_metadata.keys() else None)
    cw_data['Source'] =  cw_data['QuestionCode'].apply(lambda x:
                            cols_metadata[x][6] if x in cols_metadata.keys() else None)

    # Sector data has country specified by ISO Alpha2 while NDC are
    # listed as per ISO Alpha3, rectifying the same

    cw_data = cw_data.rename(columns = {'Country':'Alpha2'})

    countryList = pd.DataFrame(columns = ['alpha2','alpha3','name','numeric'])
    for i in pycountry.countries:
        countryList.loc[len(countryList)] = [i.alpha_2, i.alpha_3, i.name, i.numeric]
    alpha2_alpha3 = dict(zip(countryList.alpha2, countryList.alpha3))

    # using the country name from climatewatch data for homogeneity
    alpha3_country = dict(zip(tracker.ISO, tracker.Country))

    # adding ISO Alpha3 and Country names to dataframe
    cw_data['Alpha3'] = cw_data['Alpha2'].apply(lambda x:
                            alpha2_alpha3[x] if x in alpha2_alpha3.keys() else None)
    cw_data['Country'] = cw_data['Alpha3'].apply(lambda x:
                          alpha3_country[x] if x in alpha3_country.keys() else None)
    print(len(cw_data))

    # Cleaning for HTML text
    html_df = cw_data[cw_data.ResponseText.str.contains("<p>", na = False)]
    html_df = html_df.reset_index(drop=True)
    print("HTML text counts",len(html_df))
    html_df['ResponseText'] = html_df['ResponseText'].apply(lambda x:
                                          BeautifulSoup(x).get_text(separator = '|'))

    cw_data = cw_data[~cw_data.ResponseText.str.contains("<p>",na = False)]

    print("Without HTML",len(cw_data))
    cw_data = pd.concat([cw_data,html_df], ignore_index = True)
    print("Dataframe length",len(cw_data))

    # Response Text has Separator ('|', ';'), which signifies there are mutltiple response
    # text in same row. Need to split this across different rows and duplicating all
    # other columns info

    cw_data["ResponseText"]=cw_data["ResponseText"].str.split("|")
    cw_data = cw_data.explode("ResponseText").reset_index(drop=True)

    cw_data["ResponseText"]=cw_data["ResponseText"].str.split("<br>")
    cw_data = cw_data.explode("ResponseText").reset_index(drop=True)

    cw_data["ResponseText"]=cw_data["ResponseText"].str.split(";")
    cw_data = cw_data.explode("ResponseText").reset_index(drop=True)

    cw_data['ResponseText'] = cw_data.ResponseText.str.strip()
    cw_data = cw_data.dropna(subset = ['Alpha3'])

    print("Dataframe length after explode", len(cw_data))
    cw_data.drop(['GlobalCategory','GroupIndicator'], axis=1, inplace=True)

    cw_data = cw_data.drop_duplicates()
    cw_data = cw_data.reset_index(drop = True)
    cw_data['Source'] = 'CW'
    print("df length without duplicates", len(cw_data))

    cw_data['CWInfo'] = cw_data[['Alpha2','Sector', 'SubSector', 'QuestionCode',
          'OverviewCategory', 'QuestionText','QuestionDefinition']].to_dict('records')

    cw_data.drop(['Alpha2','Sector', 'SubSector', 'QuestionCode',
          'OverviewCategory', 'QuestionText','QuestionDefinition'],axis=1, inplace = True)

    cw_data['Document'] = cw_data.Document.str.strip()

    return cw_data


def getIKIResposnse(path_to_iki:str) -> Tuple[pd.DataFrame,pd.DataFrame]:
    """
    This function relies upon the "IKI Data" which has 2 important files:
      1. 20230125_NDC-Database-Analysis.xlsx
      2. code_book.xlsx


    Params
    -----------
    path_to_iki: path to the IKI data


    Return
    --------
    df_concat: Dataframe with IKI responses
    df_documents: Dataframe with list of documents relied upon to create IKI
                  responses

    """
    # Pull IKI data one tab at a time
    file_name = '20230125_NDC-Database-Analysis.xlsx'

    # read each tab and store as separate object
    df_netzero = pd.read_excel(os.path.join('..', path_to_iki, file_name),
                              sheet_name="Net-zero", index_col=None,
                              na_values=['NA'], usecols='A:Q', skiprows=7)
    df_targets = pd.read_excel(os.path.join('..', path_to_iki, file_name),
                              sheet_name="Targets", index_col=None,
                              na_values=['NA'], usecols='A:Y', skiprows=7)
    df_mitigation = pd.read_excel(os.path.join('..', path_to_iki, file_name),
                                  sheet_name="Mitigation", index_col=None,
                                  na_values=['NA'], usecols='A:AP', skiprows=8)
    df_adaptation = pd.read_excel(os.path.join('..', path_to_iki, file_name),
                                  sheet_name="Adaptation", index_col=None,
                                  na_values=['NA'], usecols='A:AN', skiprows=7)
    df_governance = pd.read_excel(os.path.join('..', path_to_iki, file_name),
                                  sheet_name="Governance", index_col=None,
                                  na_values=['NA'], usecols='A:I', skiprows=7)
    df_implementation = pd.read_excel(os.path.join('..', path_to_iki, file_name),
                                      sheet_name="Implementation", index_col=None,
                                      na_values=['NA'], usecols='A:I', skiprows=7)
    df_investment = pd.read_excel(os.path.join('..', path_to_iki, file_name),
                                  sheet_name="Investment", index_col=None,
                                  na_values=['NA'], usecols='A:I', skiprows=7)
    df_benefits = pd.read_excel(os.path.join('..', path_to_iki, file_name),
                                sheet_name="Benefits", index_col=None,
                                na_values=['NA'], usecols='A:I', skiprows=7)
    df_covid = pd.read_excel(os.path.join('..', path_to_iki, file_name),
                            sheet_name="COVID19", index_col=None, na_values=['NA'],
                            usecols='A:I', skiprows=7)
    df_documents = pd.read_excel(os.path.join('..', path_to_iki, file_name),
                                sheet_name="Document", index_col=None,
                                na_values=['NA'], usecols='A:P', skiprows=7)
    # Make the variable names machine-friendly
    file_name = 'code_book.xlsx'
    df_netzero.columns = \
        pd.read_excel(os.path.join('..', path_to_iki, file_name), sheet_name="netzero",
                      index_col=None, usecols='B')['var_name']
    df_targets.columns = \
        pd.read_excel(os.path.join('..', path_to_iki, file_name), sheet_name="targets",
                      index_col=None, usecols='B')['var_name']
    df_mitigation.columns = \
        pd.read_excel(os.path.join('..', path_to_iki, file_name), sheet_name="mitigation",
                      index_col=None, usecols='B')['var_name']
    df_adaptation.columns = \
        pd.read_excel(os.path.join('..', path_to_iki, file_name), sheet_name="adaptation",
                      index_col=None, usecols='B')['var_name']
    df_governance.columns = \
        pd.read_excel(os.path.join('..', path_to_iki, file_name), sheet_name="governance",
                      index_col=None, usecols='B')['var_name']
    df_implementation.columns = \
        pd.read_excel(os.path.join('..', path_to_iki, file_name),
                      sheet_name="implementation", index_col=None, usecols='B')['var_name']
    df_investment.columns = \
        pd.read_excel(os.path.join('..', path_to_iki, file_name), sheet_name="investment",
                      index_col=None, usecols='B')['var_name']
    df_benefits.columns = \
        pd.read_excel(os.path.join('..', path_to_iki, file_name), sheet_name="benefits",
                      index_col=None, usecols='B')['var_name']
    df_covid.columns = \
        pd.read_excel(os.path.join('..', path_to_iki, file_name), sheet_name="covid19",
                      index_col=None, usecols='B')['var_name']
    df_documents.columns = \
        pd.read_excel(os.path.join('..', path_to_iki, file_name), sheet_name="documents",
                      index_col=None, usecols='B')['var_name']
    # adding tab name to keep track
    df_netzero['IkiSheet'] = 'netzero'
    df_targets['IkiSheet'] = 'targets'
    df_mitigation['IkiSheet'] = 'mitigation'
    df_adaptation['IkiSheet'] = 'adaptation'
    df_governance['IkiSheet'] = 'governance'
    df_implementation['IkiSheet'] = 'implementation'
    df_investment['IkiSheet'] = 'investment'
    df_benefits['IkiSheet'] = 'benefits'
    df_covid['IkiSheet'] = 'covid19'

    # merge all sources into one dataframe for simplicity
    # we do this by normalizing the dataframe
    keep_cols = {'country_code', 'country', 'type_of_document',
                'content'}

    df_netzero = normalizedf(df_netzero,keep_cols,'IkiInfo')
    df_targets = normalizedf(df_targets, keep_cols, 'IkiInfo')
    df_mitigation = normalizedf(df_mitigation, keep_cols, 'IkiInfo')
    df_adaptation = normalizedf(df_adaptation, keep_cols, 'IkiInfo')
    df_governance = normalizedf(df_governance, keep_cols, 'IkiInfo')
    df_implementation = normalizedf(df_implementation, keep_cols, 'IkiInfo')
    df_investment = normalizedf(df_investment, keep_cols, 'IkiInfo')
    df_benefits = normalizedf(df_benefits, keep_cols, 'IkiInfo')
    df_covid = normalizedf(df_covid, keep_cols, 'IkiInfo')

    df_concat = pd.concat([df_netzero, df_targets,df_mitigation, df_adaptation,
                       df_governance,df_implementation,df_investment, df_benefits,
                       df_covid], ignore_index =True)

    # df_concat = df_concat.astype(str)
    df_concat['Source'] = 'IKITracs'
    print(f"Number of imported refs: {len(df_concat)}")

    df_concat.rename(columns = {'country_code':'Alpha3',
                            'type_of_document':'Document','country':'Country',
                            'content':'ResponseText'}, inplace = True)

    df_concat = df_concat.reset_index(drop=True)
    df_concat.ResponseText = df_concat.ResponseText.str.strip()
    df_concat.Document = df_concat.Document.str.strip()

    return df_concat, df_documents

def normalizedf(df:pd.DataFrame,keep_cols:set,col_name:str):
    """
    Takes a dataframe and Normalizes it, 'keep_cols' are kept intact and other
    are fed to dictionary object 'col_name' which becomes attribute of each row.

    Params
    --------------
    df: Dataframe which needs to be normalized
    keep_cols: Set of names of columns which need to be kept as it is, while
              other columns are converted into dictionary and saved in under one
              column which derives its name from param 'col_name'.
    col_name: dictionary object created using remaining columns except 'keep_cols'
              will be saved under the column col_name

    Return
    ---------------
    df: Normalized Dataframe

    """
    remove_cols = list(set(df.columns) - keep_cols)
    df[col_name] = df[remove_cols].to_dict('records')
    df.drop(remove_cols,axis=1,inplace= True)
    return df

In [ ]:
def basicReponseProcessing(data):
    """
        takes dataframe and drops the responses which are very generic or of
        not much use
    """
    data = data[~data['ResponseText'].str.contains("http",na=False)]
    # we drop some of the most common occuring responses which are generic and
    # dont reflect true response
    data= data[~((data.ResponseText == 'Not Available')|
                    (data.ResponseText == 'Other')|(data.ResponseText == 'Water')|
                    (data.ResponseText == 'Food and Nutrition Security')|
                    (data.ResponseText == 'Disaster Risk Management'))]
    data = data[data.ResponseWordcount > 0]
    data = data.reset_index(drop = True)
    print("df after some basic cleaning/processing:",len(data))

    return data

def percentile(n):
    """
    helping function to get the percentile value for aggregate/groupby
    https://stackoverflow.com/questions/17578115/pass-percentiles-to-pandas-agg-function

    """
    def percentile_(x):
        return np.percentile(x, n, method = 'median_unbiased')
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

def getCountryReponsestatistics(df):
    """
    gets the statistics of Responseword count for each country

    Params
    ------------
    df: dataframe which contains the responses from both Climate Watch and IKI
        This require to have Country Code, ResponseText and ResponseWordcount columns


    Return
    ----------------
    country_: dataframe which country response statistics
    responseLengthMean:global mean = mean of country_mean
    responseLengthMedian: global median != mean of country_median
    responseLengthMax:  ninetyfifthpercentile of country_ninetyfifthpercentile

    """
    country_ = df.groupby(['Alpha3','Country'], as_index=False).agg(
          value_count=('ResponseText','size'),mean_val=('ResponseWordcount',np.mean),
          median_val=('ResponseWordcount',np.median),min_val=('ResponseWordcount',min),
          max_val=('ResponseWordcount',max),std_val=('ResponseWordcount',np.std),
          ninetyfifth_percentile=('ResponseWordcount',percentile(95)))

    responseLengthMean = np.mean(country_.mean_val)
    responseLengthMedian = np.median(df.ResponseWordcount)

    # getting the 95 percentile value, As Max value doesnt
    # reflect good assessment
    responseLengthMax = np.percentile(country_.ninetyfifth_percentile, 95)

    # getting the aggregates at Global level.
    country_.iloc[-1] = ['Global','Global',None, responseLengthMean,
                      responseLengthMedian, None, responseLengthMax, None, None]
    country_.index = country_.index +1

    return country_ , responseLengthMean, responseLengthMedian, responseLengthMax

## Main Code

In [ ]:
# setting up the paths to climate watch data
path_to_cwdata = "/content/drive/MyDrive/Colab Notebooks/CPU/Step1/input/ClimateWatch/"
cw_responses = getCWResponse(path_to_cwdata)

# set path to iki tracs data
path_to_iki = "/content/drive/MyDrive/Colab Notebooks/CPU/Step1/input/IKI/"
iki_responses, iki_document_list = getIKIResposnse(path_to_iki)

# combine both response datasets from Climat Watch and IKI
cw_iki = pd.concat([cw_responses,iki_responses], axis=0, ignore_index= True)
cw_iki = cw_iki.reset_index(drop = True)
print("length of df:", len(cw_iki))

80867
HTML text counts 1459
Without HTML 79408
Dataframe length 80867
Dataframe length after explode 91444
df length without duplicates 66075


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Number of imported refs: 4226
length of df: 70301


In [ ]:
# getting word counts in Response for each row
cw_iki['ResponseWordcount'] = cw_iki.ResponseText.str.split().str.len()

# dropping some not useful responses
cw_iki = basicReponseProcessing(cw_iki)

# get some response word count statistics
# responseLengthMean = global mean = mean of country_mean
# responseLengthMedian = global median != mean of country_median
# responseLengthMax = ninetyfifthpercentile of country_ninetyfifthpercentile
responsestats,responseLengthMean, responseLengthMedian,responseLengthMax  = \
                                            getCountryReponsestatistics(cw_iki)

df after some basic cleaning/processing: 51060


In [ ]:
print(f"Response length Mean is {responseLengthMean}")
print(f"Response length Median is {responseLengthMedian}")
print(f"Response length Max Length is {responseLengthMax}")
display(responsestats.head())
display(cw_iki.info())

Response length Mean is 17.06963864364939
Response length Median is 10.0
Response length Max Length is 85.7875


,Alpha3,Country,value_count,mean_val,median_val,min_val,max_val,std_val,ninetyfifth_percentile
1,AFG,Afghanistan,308.0,7.974026,7.0,1.0,51.0,6.793317,20.5
2,AGO,Angola,473.0,7.452431,5.0,1.0,39.0,6.094103,21.0
3,ALB,Albania,467.0,18.171306,14.0,1.0,113.0,14.971382,46.0
4,AND,Andorra,56.0,29.571429,25.0,3.0,364.0,48.657563,57.5
5,ARE,United Arab Emirates,320.0,32.996875,30.0,2.0,118.0,21.726499,75.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51060 entries, 0 to 51059
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Document           51053 non-null  object 
 1   ResponseText       51060 non-null  object 
 2   Source             51060 non-null  object 
 3   Alpha3             51053 non-null  object 
 4   Country            51052 non-null  object 
 5   CWInfo             46973 non-null  object 
 6   IkiInfo            4087 non-null   object 
 7   ResponseWordcount  51060 non-null  float64
dtypes: float64(1), object(7)
memory usage: 3.1+ MB


None

In [ ]:
print(f"Number of rows with Response length > 85 is {sum(cw_iki.ResponseWordcount>85)}")
print(f"Number of rows with Response length > 60 is {sum(cw_iki.ResponseWordcount>60)}")

Number of rows with Response length > 85 is 366
Number of rows with Response length > 60 is 1091


In [ ]:
# comment out and run the cell to save the data
import json
path_to_step1 = "/content/drive/MyDrive/Colab Notebooks/CPU/Step1/output/"
jsonfile = cw_iki.to_json(orient = 'records')
parsed = json.loads(jsonfile)
with open(path_to_step1 +'cwiki_responses.json', 'w') as file:
    json.dump(parsed, file, indent=4)

# save documents list
jsonfile = iki_document_list.to_json(orient = 'records')
parsed = json.loads(jsonfile)
with open(path_to_step1 +'documents_list.json', 'w') as file:
    json.dump(parsed, file, indent=4)

# save country wide response statistics
jsonfile = responsestats.to_json(orient = 'records')
parsed = json.loads(jsonfile)
with open(path_to_step1 +'responsestats.json', 'w') as file:
    json.dump(parsed, file, indent=4)